# Question 5

In [ ]:
import pandas as pd
import numpy as np
!pip install arch --upgrade
from arch import arch_model

In [11]:
kospi = pd.read_csv("^kospi_d.csv", parse_dates=["Date"], index_col="Date")
ndx = pd.read_csv("^ndx_d.csv")
# kospi is longer

In [12]:
kospi = kospi.sort_index()

In [ ]:
# Resample to get Thursday closing prices (weekly)
thursday_prices = kospi.resample("W-THU").last()

# Select every second Thursday to get non-overlapping 2-week returns
non_overlapping_thursdays = thursday_prices.iloc[::2]

# Compute non-overlapping log returns
log_returns = np.log(non_overlapping_thursdays["Close"] / non_overlapping_thursdays["Close"].shift(1)).dropna()

In [ ]:
# get first 10 years
start_date = log_returns.index.min()  # Get the first available date
end_date = start_date + pd.DateOffset(years=10)  # Add 10 years to the start date

# Filter the returns to get the first 10 years
first_10_years_log_returns = log_returns[(log_returns.index >= start_date) & (log_returns.index < end_date)]

In [ ]:
# 5. Fit a GARCH(1,1) model to the initial sample
garch_model = arch_model(initial_returns, vol='Garch', p=1, q=1)
garch_fit = garch_model.fit()

# 6. Calculate the 5% 2-week VaR using the GARCH model
# Forecast the volatility for the next 2-week period
forecast = garch_fit.forecast(start=0, horizon=2)  # 2-week forecast

# Extract the volatility forecast for the 2-week horizon
volatility_forecast = forecast.variance.values[-1, 1] ** 0.5  # square root for volatility

# 7. Calculate 5% VaR (the negative quantile of the normal distribution)
VaR_5_percent = -np.percentile(np.random.normal(0, volatility_forecast, 10000), 5)

Date
1980-01-24   -0.059467
1980-02-07    0.039049
1980-02-21   -0.000955
1980-03-06   -0.017843
1980-03-20    0.018512
                ...   
1989-11-16    0.025834
1989-11-30   -0.002402
1989-12-14    0.023736
1989-12-28   -0.020003
1990-01-11   -0.007115
Name: Close, Length: 257, dtype: float64